In [0]:
# Creación Dimensión Tiempo
**Descripción**
**Descripción**
Dimensión tiempo que abarca fechas desde el 1 de enero de 2024 hasta finales de 2026.

In [0]:
from pyspark.sql.functions import col, explode, sequence, to_date, year, month, dayofmonth, date_format, quarter, regexp_replace, current_timestamp

# 1. Generar el rango de fechas base
df_dates = spark.sql("SELECT explode(sequence(to_date('2024-01-01'), to_date('2026-12-31'), interval 1 day)) as fecha")

# 2. Transformar y agregar metadatos
dim_tiempo = df_dates.select(
    # SK: Generamos el ID numérico (ej. 20240101)
    regexp_replace(col("fecha").cast("string"), "-", "").cast("int").alias("tiempo_key"), 
    col("fecha").alias("fecha_bk"), # Clave de negocio (Business Key)
    year(col("fecha")).alias("anio"),
    month(col("fecha")).alias("mes_numero"),
    date_format(col("fecha"), "MMMM").alias("mes_nombre"),
    quarter(col("fecha")).alias("trimestre"),
    dayofmonth(col("fecha")).alias("dia"),
    date_format(col("fecha"), "EEEE").alias("dia_semana"),
    # Agregamos la columna de auditoría aquí mismo
    current_timestamp().alias("fecha_carga") 
)

# 3. Guardar en el catálogo (una sola operación de escritura)
dim_tiempo.write.mode("overwrite").saveAsTable("workspace.gold.dim_tiempo")
print("✅ dim_tiempo creada.")